# AI-Powered HR Assistant: A Use Case for Nestle’s HR Policy Documents

## Nestlé HR Policy Chatbot Overview

This Jupyter Notebook implements a conversational chatbot designed to answer user queries based on information contained within Nestlé's HR policy documents. It leverages several powerful technologies from the field of Natural Language Processing (NLP) and Large Language Models (LLMs) to achieve this:

1.  **Document Loading and Chunking:** The notebook begins by loading the HR policy document (in PDF format) and splitting it into smaller, manageable text chunks. This is crucial because LLMs have limitations on the amount of text they can process at once.

2.  **Text Embeddings:** Each text chunk is then converted into a numerical vector representation called an "embedding." These embeddings capture the semantic meaning of the text, allowing the system to understand the relationships between different pieces of information. OpenAI's embedding models are used for this purpose.

3.  **Vector Database:** The embeddings are stored in a vector database (Chroma), which allows for efficient similarity search. This means that when a user asks a question, the system can quickly find the most relevant text chunks from the HR policy.

4.  **Question Answering with LLM:** The most relevant text chunks are then passed to a large language model (OpenAI's GPT model) along with the user's question. The LLM uses this context to generate a coherent and informative answer.

5.  **User Interface:** Finally, a user-friendly interface is created using Gradio, allowing users to easily interact with the chatbot.

In summary, this notebook demonstrates a complete workflow for building a question-answering system over a PDF document using state-of-the-art NLP and LLM techniques. This approach can be generalized to other document types and domains, making it a valuable tool for information retrieval and knowledge management. The notebook is structured with Markdown explanations and code blocks separated by functionality, making it easy to follow and understand the implementation details.